In [ ]:
import cv2
import numpy as np
import os

def f_mlo(image):
    # 读取图像
    # image = cv2.imread(path, cv2.IMREAD_GRAYSCALE)

    # 裁剪图像：去除左右上下各5像素
    # cropped_image = image[5:-5, 5:-5]
    cropped_image = image
    # 获取裁剪后图像的高度和宽度
    height, width = cropped_image.shape[:2]
    left_half = cropped_image[:, :width // 2]
    right_half = cropped_image[:, width // 2:]
    left_sum = cv2.sumElems(left_half)[0]
    right_sum = cv2.sumElems(right_half)[0]
    left_true = (left_sum > right_sum)
    # 分别获取图像的上、下、左、右边界像素
    top_pixels = cropped_image[0, :]
    bottom_pixels = cropped_image[height - 1, :]
    left_pixels = cropped_image[:, 0]
    right_pixels = cropped_image[:, width - 1]

    # 统计非黑像素数量
    top_nonzero = cv2.countNonZero(top_pixels)
    bottom_nonzero = cv2.countNonZero(bottom_pixels)
    left_nonzero = cv2.countNonZero(left_pixels)
    right_nonzero = cv2.countNonZero(right_pixels)

    # 计算边界长度
    top_length = width
    bottom_length = width
    left_length = height
    right_length = height

    # 检查非黑像素数量是否超过边界长度的 1/8
    threshold = 1 / 8

    count = 0

    if top_nonzero > top_length * threshold:
        count += 1

    if bottom_nonzero > bottom_length * threshold:
        count += 1

    if left_nonzero > left_length * threshold:
        count += 1

    if right_nonzero > right_length * threshold:
        count += 1

    # 如果有两个以上的计数器超过 1，输出图片名
    if count >= 2 and left_true:
        return "lmlo"
    elif count >= 2 and not left_true:
        return "rmlo"
    elif count < 2 and left_true:
        return "lcc"
    elif count < 2 and not left_true:
        return "rcc"

image_in_path=r'/data_hs/sjwlab/meixuewen/project2/dataset/localduo_data/images'
image_out_path=r'/data_hs/sjwlab/meixuewen/project2/dataset/localduo_data/images_two_many'
os.makedirs(image_out_path,exist_ok=True)

for filename in os.listdir(image_in_path):
    image_path=os.path.join(image_in_path,filename)
    out_path1=os.path.join(image_out_path,filename)
    os.makedirs(out_path1,exist_ok=True)
    for img in os.listdir(image_path):
        impath=os.path.join(image_path,img)
        im=cv2.imread(impath,0)
        # rgb_image = cv2.cvtColor(im, cv2.COLOR_GRAY2BGR)
        expand=f_mlo(im)[1:]
        out_path2=os.path.join(out_path1,expand)
        os.makedirs(out_path2,exist_ok=True)
        cv2.imwrite(os.path.join(out_path2,img),im)
  




多线程

In [ ]:
import cv2
import numpy as np
import os
import concurrent.futures
from threading import Lock

lock = Lock()  # 创建一个锁对象

def f_mlo(image):
    # 裁剪图像：去除左右上下各5像素
    cropped_image = image
    height, width = cropped_image.shape[:2]
    left_half = cropped_image[:, :width // 2]
    right_half = cropped_image[:, width // 2:]
    left_sum = cv2.sumElems(left_half)[0]
    right_sum = cv2.sumElems(right_half)[0]
    left_true = (left_sum > right_sum)

    top_pixels = cropped_image[0, :]
    bottom_pixels = cropped_image[height - 1, :]
    left_pixels = cropped_image[:, 0]
    right_pixels = cropped_image[:, width - 1]

    top_nonzero = cv2.countNonZero(top_pixels)
    bottom_nonzero = cv2.countNonZero(bottom_pixels)
    left_nonzero = cv2.countNonZero(left_pixels)
    right_nonzero = cv2.countNonZero(right_pixels)

    top_length = width
    bottom_length = width
    left_length = height
    right_length = height

    threshold = 1 / 8

    count = sum([
        top_nonzero > top_length * threshold,
        bottom_nonzero > bottom_length * threshold,
        left_nonzero > left_length * threshold,
        right_nonzero > right_length * threshold
    ])

    if count >= 2 and left_true:
        return "lmlo"
    elif count >= 2 and not left_true:
        return "rmlo"
    elif count < 2 and left_true:
        return "lcc"
    elif count < 2 and not left_true:
        return "rcc"

def process_image(impath, out_path1):
    im = cv2.imread(impath, 0)
    expand = f_mlo(im)[1:]
    out_path2 = os.path.join(out_path1, expand)
    os.makedirs(out_path2, exist_ok=True)
    
    # 在写入文件前加锁
    with lock:
        print(out_path2)
        cv2.imwrite(os.path.join(out_path2, os.path.basename(impath)), im)

if __name__ == "__main__":
    image_in_path = r'/data_hs/sjwlab/meixuewen/project2/dataset/localduo_data/myim2'
    image_out_path = r'/data_hs/sjwlab/meixuewen/project2/dataset/localduo_data/myim_twomany'
    os.makedirs(image_out_path, exist_ok=True)

    tasks = []

    for filename in os.listdir(image_in_path):
        image_path = os.path.join(image_in_path, filename)
        out_path1 = os.path.join(image_out_path, filename)
        os.makedirs(out_path1, exist_ok=True)
        
        # 遍历子文件夹内的图像文件
        for img in os.listdir(image_path):
            impath = os.path.join(image_path, img)
            tasks.append((impath, out_path1))

    # 使用线程池执行任务
    with concurrent.futures.ThreadPoolExecutor() as executor:
        futures = [executor.submit(process_image, *task) for task in tasks]
        
        # 等待所有任务完成
        concurrent.futures.wait(futures)

重新优化

In [ ]:
# import cv2
# import numpy as np
# import os

# def f_mlo(image):
#     # 读取图像
#     # image = cv2.imread(path, cv2.IMREAD_GRAYSCALE)

#     # 裁剪图像：去除左右上下各5像素
#     # cropped_image = image[5:-5, 5:-5]
#     cropped_image = image
#     # 获取裁剪后图像的高度和宽度
#     height, width = cropped_image.shape[:2]
#     left_half = cropped_image[:, :width // 2]
#     right_half = cropped_image[:, width // 2:]
#     left_sum = cv2.sumElems(left_half)[0]
#     right_sum = cv2.sumElems(right_half)[0]
#     left_true = (left_sum > right_sum)
#     # 分别获取图像的上、下、左、右边界像素
#     top_pixels = cropped_image[0, :]
#     bottom_pixels = cropped_image[height - 1, :]
#     left_pixels = cropped_image[:, 0]
#     right_pixels = cropped_image[:, width - 1]

#     # 统计非黑像素数量
#     top_nonzero = cv2.countNonZero(top_pixels)
#     bottom_nonzero = cv2.countNonZero(bottom_pixels)
#     left_nonzero = cv2.countNonZero(left_pixels)
#     right_nonzero = cv2.countNonZero(right_pixels)

#     # 计算边界长度
#     top_length = width
#     bottom_length = width
#     left_length = height
#     right_length = height

#     # 检查非黑像素数量是否超过边界长度的 1/8
#     threshold = 1 / 8

#     count = 0

#     if top_nonzero > top_length * threshold:
#         count += 1

#     if bottom_nonzero > bottom_length * threshold:
#         count += 1

#     if left_nonzero > left_length * threshold:
#         count += 1

#     if right_nonzero > right_length * threshold:
#         count += 1

#     # 如果有两个以上的计数器超过 1，输出图片名
#     if count >= 2 and left_true:
#         return "lmlo"
#     elif count >= 2 and not left_true:
#         return "rmlo"
#     elif count < 2 and left_true:
#         return "lcc"
#     elif count < 2 and not left_true:
#         return "rcc"

# def chest_structure_detection(image):
#     gray = image
#     blurred = cv2.GaussianBlur(gray, (5, 5), 0)
#     _, thresh = cv2.threshold(blurred, 60, 255, cv2.THRESH_BINARY)
#     contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
#     for contour in contours:
#         area = cv2.contourArea(contour)
#         if area > (image.shape[0] * image.shape[1]) * 0.05:  # 大轮廓可能是胸部结构
#             return 'MLO'
#     return 'CC'

# cc=0
# mlo=0
# im_path=r'/data_hs/sjwlab/meixuewen/project2/dataset/redealname/CR2546262LEFT'
# for im in os.listdir(im_path):
#     inpath=os.path.join(im_path,im)
#     img=cv2.imread(inpath,0)
#     flag=chest_structure_detection(img)
#     print(flag)
#     if flag=='CC':
#         cc+=1
#     elif flag=='MLO':
#         mlo+=1
# print(cc,mlo)



import cv2
import os


im_path=r'/data_hs/sjwlab/meixuewen/project2/dataset/redealname/CR2280821RIGHT/DXm.1.2.392.200036.9125.4.0.890226153.654312808.264908388.png'

img=cv2.imread(im_path,0)
img[img>=30]=255
cv2.imwrite('test.png',img)


In [34]:
####对称性差异性


import cv2
import numpy as np

def check_image_symmetry(image):
    # 将图像转换为灰度图
    gray = image
    
    # 计算图像的水平翻转
    flipped = cv2.flip(gray, 1)
    
    # 计算原始图像和翻转图像之间的差异
    diff = cv2.absdiff(gray, flipped)
    
    # 对差异图像进行二值化处理
    _, thresh = cv2.threshold(diff, 30, 255, cv2.THRESH_BINARY)
    
    # 计算非零像素的数量，即差异的总和
    non_zero_pixels = np.count_nonzero(thresh)
    
    # 计算对称性得分（差异像素占总像素的比例）
    symmetry_score = non_zero_pixels / (gray.shape[0] * gray.shape[1])
    
    return symmetry_score

# 使用示例
image_path = '/data_hs/sjwlab/meixuewen/project2/dataset/redealname/CR2456712LEFT/DXm.1.2.392.200036.9125.4.0.17773857.641074536.264908388.png'
image = cv2.imread(image_path)
image[image>=30]=255
# 检查图像对称性
score = check_image_symmetry(image)
print(f"Symmetry score: {score}")


Symmetry score: 1.492181429911782


In [ ]:
###批量化

import cv2
import numpy as np

file_path=r'/data_hs/sjwlab/meixuewen/project2/dataset/redealname'
out_path=r'/data_hs/sjwlab/meixuewen/project2/dataset/redealname_view'
os.makedirs(out_path,exist_ok=True)
sum=0
n=0

# for idfile  in os.listdir(file_path):
#     idpath=os.path.join(file_path,idfile)
#     out=os.path.join(out_path,idfile)
#     os.makedirs(out,exist_ok=True)
#     for im in os.listdir(idpath):
#         impath=os.path.join(idpath,im)
#         image = cv2.imread(impath)
#         image[image>=30]=255
#         score = check_image_symmetry(image)
#         sum=sum+score
#         n+=1
#     average=sum/n
#     for im2 in os.listdir(idpath):
#         impath2=os.path.join(idpath,im2)
#         image2 = cv2.imread(impath2)
#         baoliu=image2
#         image2[image2>=30]=255
#         score = check_image_symmetry(image2)
#         if score>=average:
#             out1=os.path.join(out,'cc')         
#             os.makedirs(out1,exist_ok=True)
#             cv2.imwrite(os.path.join(out1,im2),baoliu) 
#         elif score<=average:
#             out2=os.path.join(out,'mlo')
#             os.makedirs(out2,exist_ok=True)
#             cv2.imwrite(os.path.join(out2,im2),baoliu)   
# 
for idfile in os.listdir(file_path):
    idpath = os.path.join(file_path, idfile)
    out = os.path.join(out_path, idfile)
    os.makedirs(out, exist_ok=True)
    for im in os.listdir(idpath):
        impath = os.path.join(idpath, im)
        image = cv2.imread(impath)
        original_image = image.copy()  # 保存原始图像的副本
        # 对图像进行预处理（如果需要的话）
        gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        _, thresh = cv2.threshold(gray, 30, 255, cv2.THRESH_BINARY)
        score = check_image_symmetry(thresh)
        sum += score
        n += 1
    average = sum / n
    for im in os.listdir(idpath):
        impath = os.path.join(idpath, im)
        image = cv2.imread(impath)
        original_image = image.copy()  # 保存原始图像的副本
        gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        _, thresh = cv2.threshold(gray, 30, 255, cv2.THRESH_BINARY)
        score = check_image_symmetry(thresh)
        if score >= average:
            out1 = os.path.join(out, 'cc')
            os.makedirs(out1, exist_ok=True)
            cv2.imwrite(os.path.join(out1, im), original_image)  # 保存原始图像
        else:
            out2 = os.path.join(out, 'mlo')
            os.makedirs(out2, exist_ok=True)
            cv2.imwrite(os.path.join(out2, im), original_image)  # 保存原始图像 


速度优化

In [35]:
import cv2
import numpy as np
import os
from concurrent.futures import ProcessPoolExecutor, as_completed

file_path = r'/data_hs/sjwlab/meixuewen/project2/dataset/orglocal/images_detxt'
out_path = r'/data_hs/sjwlab/meixuewen/project2/dataset/orglocal/images_detxt_view'
os.makedirs(out_path, exist_ok=True)

def process_directory(idpath, out_path):
    out_dir = os.path.join(out_path, os.path.basename(idpath))
    os.makedirs(out_dir, exist_ok=True)
    
    scores = []
    images = []

    for im in os.listdir(idpath):
        impath = os.path.join(idpath, im)
        image = cv2.imread(impath)
        original_image = image.copy()
        gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        _, thresh = cv2.threshold(gray, 30, 255, cv2.THRESH_BINARY)
        score = check_image_symmetry(thresh)
        scores.append(score)
        images.append((original_image, im))

    average = np.mean(scores)
    
    for (original_image, im), score in zip(images, scores):
        if score >= average:
            out1 = os.path.join(out_dir, 'cc')
            os.makedirs(out1, exist_ok=True)
            cv2.imwrite(os.path.join(out1, im), original_image)
        else:
            out2 = os.path.join(out_dir, 'mlo')
            os.makedirs(out2, exist_ok=True)
            cv2.imwrite(os.path.join(out2, im), original_image)

def main():
    with ProcessPoolExecutor() as executor:
        futures = [executor.submit(process_directory, os.path.join(file_path, idfile), out_path) for idfile in os.listdir(file_path)]
        for future in as_completed(futures):
            future.result()

if __name__ == '__main__':
    main()
    